In [32]:
import nltk
import wikipedia

In [33]:
def named_entity_recognition(text):
    data = nltk.word_tokenize(text)
    data = nltk.pos_tag(data)
    data = nltk.ne_chunk(data)
    data = list(data)
    
    final = False
    
    for x in data:
        if type(x) is nltk.tree.Tree:
            final = x[0][0]
            
    return final

In [34]:
d = named_entity_recognition("Can you give me some info about Cahrles")

In [35]:
wikipedia.search("Bulgaria")

['Bulgaria',
 'Demographics of Bulgaria',
 'Sofia',
 'Bulgaria Air',
 "People's Republic of Bulgaria",
 'Balkan Wars',
 'Volga Bulgaria',
 'List of Bulgarian consorts',
 'Georgi Markov',
 'List of Bulgarian monarchs']

In [36]:
bg = wikipedia.page("Bulgaria")

In [39]:
bg.categories

['All articles containing potentially dated statements',
 'All articles needing additional references',
 'All articles with dead external links',
 'All articles with unsourced statements',
 'All articles with vague or ambiguous time',
 'Articles containing Bulgarian-language text',
 'Articles containing potentially dated statements from 2012',
 'Articles including recorded pronunciations (English)',
 'Articles needing additional references from March 2018',
 'Articles with Bulgarian-language external links',
 'Articles with Curlie links',
 'Articles with dead external links from December 2017',
 'Articles with hAudio microformats',
 'Articles with permanently dead external links',
 'Articles with unsourced statements from October 2011',
 'Balkan countries',
 'Bulgaria',
 'CS1 Bulgarian-language sources (bg)',
 'CS1 French-language sources (fr)',
 'CS1 German-language sources (de)',
 'Coordinates on Wikidata',
 'Former empires',
 'Good articles',
 'Member states of NATO',
 'Member state

In [37]:
len(bg.content[:1*10**3])

1000

In [38]:
len(bg.content)

52454